In [1]:
from Swarm import Swarm
import Helper

import sys
import copy
import time
import argparse
import os

import numpy as np
from scipy import stats

from multiprocessing import Pool
from multiprocessing import cpu_count
from itertools import repeat
from tqdm import tqdm

from Helper import Write_Log

In [2]:
def lossFunction(tar,b,lossFunc = 0):
    if (lossFunc == 1):
        newCost = np.sum( (b-tar)**2, axis=1 )/len(b)#MSE
    elif(lossFunc == 2):
        newCost = np.sqrt(np.sum( (b-tar)**2, axis=1 ))#RMSE
    elif(lossFunc == 3):
        #Heuber
        delta = 0.1
        y = tar
        yHat = b
        newCost = np.sum(np.where(np.abs(y-yHat) < delta,.5*(y-yHat)**2 , delta*(np.abs(y-yHat)-0.5*delta)))
    else :
        newCost = np.sum( (b-tar)**2)#SSE
    return newCost

In [3]:
# Prints statistics of the current swarm
def Print_Stats(swarm, contact, pointCount, i, outFilePtr, convFact):
    pers = stats.pearsonr(swarm.gBest[2], contact[:,3])
    spear = stats.spearmanr(swarm.gBest[2], contact[:,3])
    spearIF = stats.spearmanr(swarm.gBest[2], contact[:,2])

    error = np.sqrt( (1/pointCount) * np.sum( (swarm.gBest[2]-contact[:,3])**2 ) )

    #print('id: ' + str(swarm.id) + 
    #    ' itt: ' + str(i) + 
    #    ' Cost: ' + str(swarm.gBest[1]) + 
    #    ' Pearson: ' + str(pers[0]) + 
    #    ' Spearmen: ' + str(spear[0]) +
    #    ' IFSpear: ' + str(spearIF[0]) +
    #    ' error: ' + str(error))
    thisOutFilePtr = 'outputFolder/'+outFilePtr +str(convFact)
    

def Write_Stats(swarm, contact, outFilePtr):
    Helper.Write_Output(outFilePtr, swarm.gBest[0])

# Performs one operation and prints statistics of current swarm
def One_Move(ittCount, swarm, contact, pointCount, threshold,  outFilePtr, convFact):
    saveGBestCost = float('inf')
    totTime = 0


    for i in range(ittCount):
        if (i%1000 == 0) and (swarm.gBest is not None):
            #error = np.sqrt( (1/pointCount) * np.sum( (swarm.gBest[2]-contact[:,3])**2 ) )
            error = lossFunction(contact[:,3],swarm.gBest[2])#np.sum( (swarm.gBest[2]-contact[:,3])**2 )
            Print_Stats(swarm, contact, pointCount, i, outFilePtr, convFact)
            
                

            if (np.abs(saveGBestCost - error)) >= threshold:
                saveGBestCost = error
            else:
                return i, totTime

        operation(i, swarm)


    return i

# Performs a single PSO pass: Velocity calculation, update position, get new cost
def operation(i, swarm):
    swarm.Calc_Vel(ittCount,i)
    swarm.Update_Pos(i)
    swarm.Cost()

# Optimizes single swarm
def Optimize(inFilePtr, outFilePtr, convFact,constraint,points,zeroInd,lossFunc = 0):
    dist = 1.0 / (constraint[:,2]**convFact)
    constraint = np.insert(constraint,3, dist ,axis=1)
    
    swarm = Swarm(constraint, len(points), randVal=randRange, swarmSize=swarmSize, zeroInd=zeroInd,lossFunc = lossFunc )

    ittFin = One_Move(ittCount, swarm, constraint, len(points), threshold,  outFilePtr, convFact)
    
    #pbar.update(1)
    return (stats.pearsonr(swarm.gBest[2], constraint[:,3])[0], 
                    stats.spearmanr(swarm.gBest[2], constraint[:,3])[0], 
                    lossFunction(constraint[:,3],swarm.gBest[2]),
                    ittFin,
                     swarm.id, swarm)

# Runs in paralel if passed multiple rangeSpace
def Par_Choice(inFilePtr, outFilePtr, alpha,lossFunc = 0):
    contact, points, zeroInd = Helper.Read_Data(inFilePtr, alpha)
    
    bestSwarm = None
    if 1==1:
        convStore = []
        alphas = np.array(range(int(alpha[0]),int(alpha[1]),int(alpha[2])))/100
        pool = Pool(processes=PROC_COUNT)
        #pbar = tqdm(total=len(alphas))#progress bar
        swarms = pool.starmap(Optimize,  zip(repeat(inFilePtr), repeat(outFilePtr), 
                                             alphas, 
                                             repeat(contact), repeat(points), repeat(zeroInd), repeat(lossFunc)))

        pool.close()
        pool.join()

        #swarms = sorted(swarms, key=lambda x: x[1])
        
        iforapl = 0
        for swarm in swarms:
            print(str(swarm[-1]) + ' ' + str(swarm[1]))
            contact = np.insert(contact,3, 1.0 / (contact[:,2]**alphas[iforapl]) ,axis=1)
            thisOutFile = 'outputFolder/'+outFilePtr+"_alpha_"+str(alphas[iforapl])
            
            Write_Stats(swarm[len(swarm)-1], contact, thisOutFile)
            convStore.append(swarm)
            if (bestSwarm is None) or (swarm[1] > bestSwarm[1]):
                bestSwarm = swarm
                swarmForPDB = swarm[len(swarm)-1]
                bestAlpha = alphas[iforapl]
            iforapl += 1
    else:#single thread
        bestSwarm = Optimize( inFilePtr, outFilePtr, alpha)
    contact = np.insert(contact,3, 1.0 / (contact[:,2]**bestAlpha) ,axis=1)
    outFilePtr = 'outputFolder/'+outFilePtr+"_best"
    print(bestSwarm)
    Write_Stats(swarmForPDB, contact, outFilePtr)

    return bestSwarm

def Full_List( inputFilePtr, outFilePtr , alpha, lossFunc = 0):
    convStore = []
    
    convStore.append(Par_Choice( inputFilePtr, outFilePtr, alpha, lossFunc))
    print("pearson:" + str(convStore[0][0]) + " spearman:"+
          str(convStore[0][1]) + " rmse:" + str(convStore[0][2]))

    #Helper.Write_List(convStore, outFilePtr)
    return convStore

In [4]:
sys.setrecursionlimit(10000)
PROC_COUNT = cpu_count()

rangeSpace = [] # Max scaling factor. Needs to be optimized for each specific dataset. Use two values [one, two] to multithread through a range of those two values at a interval of 5000


# Arguments for running program
# python3 ParticleChromo3D.py <input_data> <other_parameter>

randRange = 1.0
swarmSize = 15
ittCount = 30000
threshold = 0.000001

if len(rangeSpace) == 0:
    rangeSpace.append(20000)

if len(rangeSpace) > 2 and (rangeSpace[0] == rangeSpace[1]):
    rangeSpace.pop()
    
if not os.path.exists('outputFolder'):
    os.makedirs('outputFolder')



theseAlphas = np.array([0.1, 2.0, 0.2])*100
theAlphas = np.array(range(int(theseAlphas[0]),int(theseAlphas[1]),int(theseAlphas[2])))/100



In [5]:
#Gm12878 tests
lossFunctionChoice = 3 #0 = sse 1 == MSE 2 = rmse 3 == Huber

nettimes = []
allSpears = [] 
for i in range(23):
    chromosomeRunning = i+1
    #1mb
    #inFilePtr = '../input-and-models/Input/GM12878_input/KR_1mb/chr'+str(chromosomeRunning)+'_matrix.txt'
    #500kb
    inFilePtr = '../input-and-models/Input/GM12878_input/KR_500kb/chr'+str(chromosomeRunning)+'_matrix.txt'
    outFilePtr = './lossFunc/chr'+str(chromosomeRunning)

    print(inFilePtr)

    fout = inFilePtr + ".stripped"
    clean_lines = []
    f= open(inFilePtr, "r")
    lines = f.readlines()
    for l in lines:
        res = str(" ".join(l.split()))
        clean_lines.append(res)
    f.close()

    with open(fout, "w") as f:
        f.writelines('\n'.join(clean_lines))
    f.close()


    start = time.time()
    print("Seconds since epoch =", start)
    outputOfSwarm = Full_List( inFilePtr+".stripped", outFilePtr, theseAlphas, lossFunctionChoice)[0]
    print(outputOfSwarm)

    bestSpearm = outputOfSwarm[1]
    bestCost = outputOfSwarm[2]
    bestAlpha = theAlphas[outputOfSwarm[4]]
    bestPearsonRHO = outputOfSwarm[0]


    print("Input file: ", inFilePtr)
    print("Convert factor:: ",bestAlpha)
    print("SSE at best spearman : ", bestCost)    
    print("Best Spearman correlation Dist vs. Reconstructed Dist  : ", bestSpearm) 
    print("Best Pearson correlation Dist vs. Reconstructed Dist: ", bestPearsonRHO) 
    Write_Log("outputFolder/bestAlpha.log", inFilePtr, bestAlpha, bestCost, bestSpearm, bestPearsonRHO)
    net = time.time() - start
    allSpears.append(bestSpearm)
    nettimes.append(net)
    print("time : ", net)
    
    print("All spears : ", allSpears)
    

../input-and-models/Input/GM12878_input/KR_500kb/chr1_matrix.txt
Seconds since epoch = 1619060865.967656
<Swarm.Swarm object at 0x7f18e2e6d640> 0.8410702940601211
<Swarm.Swarm object at 0x7f18e2e6dfa0> 0.9427042384903032
<Swarm.Swarm object at 0x7f18e2e6de80> 0.9370822889497383
<Swarm.Swarm object at 0x7f18e2e6d340> 0.8924252496731604
<Swarm.Swarm object at 0x7f18e2e6d2e0> 0.8391795165995247
<Swarm.Swarm object at 0x7f18e2e6df40> 0.586077562616694
<Swarm.Swarm object at 0x7f18e2e6d2b0> 0.37961431310047694
<Swarm.Swarm object at 0x7f18e2e6dd90> 0.08666389876342183
<Swarm.Swarm object at 0x7f18e2e6dd60> 0.04363003565264511
<Swarm.Swarm object at 0x7f18e2e6df70> 0.03622487382615239
(0.9307776404544553, 0.9427042384903032, 106.9122270740676, 29999, 1, <Swarm.Swarm object at 0x7f18e2e6dfa0>)
pearson:0.9307776404544553 spearman:0.9427042384903032 rmse:106.9122270740676
(0.9307776404544553, 0.9427042384903032, 106.9122270740676, 29999, 1, <Swarm.Swarm object at 0x7f18e2e6dfa0>)
Input file:  .

In [6]:
for i in allSpears:
    print(i)
    
print("Now times")
for i in nettimes:
    print(i)

0.9427042384903032
0.9425343117726278
0.9522384910721239
0.9641581654494866
0.9646248273309491
0.9530803150456939
0.9330850240513907
0.9588537091254616
0.938041996575248
0.9412759933854884
0.9442950031626112
0.9511391133296822
0.9500188266067295
0.943720883964746
0.9379405988202568
0.9342686349787163
0.926116623595097
0.9205360973271515
0.9354403527108366
0.940422994496439
0.9537557802309967
0.9410034464229079
0.9724889806099225
Now times
6514.744927883148
7518.421190977097
3594.694443464279
3323.508765220642
2899.6714050769806
2585.2554848194122
2199.7680633068085
1816.1158261299133
1191.199025630951
1515.3830788135529
1491.5861523151398
1445.9175927639008
646.1340754032135
489.65967321395874
381.12022709846497
330.4406855106354
312.39058232307434
260.9833388328552
85.36009311676025
109.70872259140015
36.53969645500183
32.38657355308533
2086.8045423030853


In [5]:
#Synthetic data set
lossFunctionChoice = 3 #0 = sse 1 == MSE 2 = rmse 3 == Huber

allSpears = [] 

inFilePtr = '../input-and-models/Input/Synthetic/chainDres5_Matrix_noise000.txt'
#inFilePtr = '../input-and-models/Input/GM12878_input/KR_1mb/chr'+str(chromosomeRunning)+'_matrix.txt'
outFilePtr = './lossFunc/synth_loss_mse'

print(inFilePtr)

fout = inFilePtr + ".stripped"
clean_lines = []
f= open(inFilePtr, "r")
lines = f.readlines()
for l in lines:
    res = str(" ".join(l.split()))
    clean_lines.append(res)
f.close()

with open(fout, "w") as f:
    f.writelines('\n'.join(clean_lines))
f.close()


start = time.time()
print("Seconds since epoch =", start)
outputOfSwarm = Full_List( inFilePtr+".stripped", outFilePtr, theseAlphas, lossFunctionChoice)[0]
print(outputOfSwarm)

bestSpearm = outputOfSwarm[1]
bestCost = outputOfSwarm[2]
bestAlpha = theAlphas[outputOfSwarm[4]]
bestPearsonRHO = outputOfSwarm[0]


print("Input file: ", inFilePtr)
print("Convert factor:: ",bestAlpha)
print("SSE at best spearman : ", bestCost)    
print("Best Spearman correlation Dist vs. Reconstructed Dist  : ", bestSpearm) 
print("Best Pearson correlation Dist vs. Reconstructed Dist: ", bestPearsonRHO) 
Write_Log("outputFolder/bestAlpha.log", inFilePtr, bestAlpha, bestCost, bestSpearm, bestPearsonRHO)
net = time.time() - start
allSpears.append(bestSpearm)
print("time : ", net)

print("All spears : ", allSpears)


../input-and-models/Input/Synthetic/chainDres5_Matrix_noise000.txt
Seconds since epoch = 1618803542.375461
<Swarm.Swarm object at 0x7f9d7bb5ea30> 0.7278820998177046
<Swarm.Swarm object at 0x7f9d7bb5e880> 0.9040950467201744
<Swarm.Swarm object at 0x7f9d7bb5ea60> 0.9648020862716763
<Swarm.Swarm object at 0x7f9d7bb5eaf0> 0.9914152815601925
<Swarm.Swarm object at 0x7f9d7bb5e8b0> 0.9938711468805349
<Swarm.Swarm object at 0x7f9d7bb5e100> 0.9982966346771843
<Swarm.Swarm object at 0x7f9d7bb5e910> 0.9929652104022952
<Swarm.Swarm object at 0x7f9d7bb5e940> 0.9835973748822087
<Swarm.Swarm object at 0x7f9d7bb5eac0> 0.9716995627372357
<Swarm.Swarm object at 0x7f9d7bb65760> 0.9592087686273185
(0.9983742471323297, 0.9982966346771843, 257.0658755574808, 29999, 1, <Swarm.Swarm object at 0x7f9d7bb5e100>)
pearson:0.9983742471323297 spearman:0.9982966346771843 rmse:257.0658755574808
(0.9983742471323297, 0.9982966346771843, 257.0658755574808, 29999, 1, <Swarm.Swarm object at 0x7f9d7bb5e100>)
Input file:  ..